# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 1 2022 12:00AM,238482,10,8086608,Eywa Pharma Inc.,Released
1,Jun 1 2022 12:00AM,238482,10,8086609,Eywa Pharma Inc.,Released
2,Jun 1 2022 12:00AM,238482,10,8086704,Eywa Pharma Inc.,Executing
3,Jun 1 2022 12:00AM,238482,10,8087388,Eywa Pharma Inc.,Executing
4,Jun 1 2022 12:00AM,238482,10,8087390,Eywa Pharma Inc.,Executing
5,Jun 1 2022 12:00AM,238482,10,8087393,Eywa Pharma Inc.,Released
6,Jun 1 2022 12:00AM,238482,10,8087394,Eywa Pharma Inc.,Executing
7,Jun 1 2022 12:00AM,238482,10,8087395,Eywa Pharma Inc.,Executing
8,Jun 1 2022 12:00AM,238482,10,8087397,Eywa Pharma Inc.,Released
9,Jun 1 2022 12:00AM,238482,10,8087399,Eywa Pharma Inc.,Executing


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
36,238486,Released,1
37,238487,Executing,1
38,238487,Released,3
39,238488,Released,2
40,238489,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Cancelled,Completed,Executing,Released
id,,,,
238484,NaN,NaN,NaN,1.0
238486,NaN,NaN,NaN,1.0
238487,NaN,NaN,1.0,3.0
238488,NaN,NaN,NaN,2.0
238489,NaN,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Cancelled,Completed,Executing,Released
id,,,,
238334,0.0,13.0,1.0,0.0
238341,0.0,0.0,0.0,35.0
238342,0.0,0.0,0.0,1.0
238343,0.0,0.0,0.0,1.0
238347,0.0,0.0,9.0,27.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Cancelled,Completed,Executing,Released
id,,,,
238334,0.0,13,1,0
238341,0.0,0,0,35
238342,0.0,0,0,1
238343,0.0,0,0,1
238347,0.0,0,9,27


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Cancelled,Completed,Executing,Released
0,238334,0.0,13,1,0
1,238341,0.0,0,0,35
2,238342,0.0,0,0,1
3,238343,0.0,0,0,1
4,238347,0.0,0,9,27


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Cancelled,Completed,Executing,Released
0,238334,0.0,13,1,
1,238341,0.0,,,35
2,238342,0.0,,,1
3,238343,0.0,,,1
4,238347,0.0,,9,27


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id             int64
Cancelled    float64
Completed     object
Executing     object
Released      object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 1 2022 12:00AM,238482,10,Eywa Pharma Inc.
19,Jun 1 2022 12:00AM,238487,10,Eywa Pharma Inc.
23,Jun 1 2022 12:00AM,238489,10,Eywa Pharma Inc.
24,May 31 2022 5:03PM,238488,19,VITABIOTICS USA INC.
26,May 31 2022 4:36PM,238486,16,American International Chemical
27,May 31 2022 3:53PM,238484,19,Eli Lilly and Company
28,May 31 2022 3:51PM,238483,102,"Senseonics, Incorporated"
29,May 31 2022 3:27PM,238481,22,"NBTY Global, Inc."
30,May 31 2022 3:26PM,238480,22,"NBTY Global, Inc."
31,May 31 2022 2:10PM,238457,20,"ACI Healthcare USA, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Cancelled,Completed,Executing,Released
0,Jun 1 2022 12:00AM,238482,10,Eywa Pharma Inc.,0.0,,12,7
1,Jun 1 2022 12:00AM,238487,10,Eywa Pharma Inc.,0.0,,1,3
2,Jun 1 2022 12:00AM,238489,10,Eywa Pharma Inc.,0.0,,,1
3,May 31 2022 5:03PM,238488,19,VITABIOTICS USA INC.,0.0,,,2
4,May 31 2022 4:36PM,238486,16,American International Chemical,0.0,,,1
5,May 31 2022 3:53PM,238484,19,Eli Lilly and Company,0.0,,,1
6,May 31 2022 3:51PM,238483,102,"Senseonics, Incorporated",0.0,,,1
7,May 31 2022 3:27PM,238481,22,"NBTY Global, Inc.",0.0,,,1
8,May 31 2022 3:26PM,238480,22,"NBTY Global, Inc.",0.0,,,1
9,May 31 2022 2:10PM,238457,20,"ACI Healthcare USA, Inc.",0.0,,1,28


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 1 2022 12:00AM,238482,10,Eywa Pharma Inc.,7,12,
1,Jun 1 2022 12:00AM,238487,10,Eywa Pharma Inc.,3,1,
2,Jun 1 2022 12:00AM,238489,10,Eywa Pharma Inc.,1,,
3,May 31 2022 5:03PM,238488,19,VITABIOTICS USA INC.,2,,
4,May 31 2022 4:36PM,238486,16,American International Chemical,1,,
5,May 31 2022 3:53PM,238484,19,Eli Lilly and Company,1,,
6,May 31 2022 3:51PM,238483,102,"Senseonics, Incorporated",1,,
7,May 31 2022 3:27PM,238481,22,"NBTY Global, Inc.",1,,
8,May 31 2022 3:26PM,238480,22,"NBTY Global, Inc.",1,,
9,May 31 2022 2:10PM,238457,20,"ACI Healthcare USA, Inc.",28,1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 1 2022 12:00AM,238482,10,Eywa Pharma Inc.,7,12,
1,Jun 1 2022 12:00AM,238487,10,Eywa Pharma Inc.,3,1,
2,Jun 1 2022 12:00AM,238489,10,Eywa Pharma Inc.,1,,
3,May 31 2022 5:03PM,238488,19,VITABIOTICS USA INC.,2,,
4,May 31 2022 4:36PM,238486,16,American International Chemical,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 1 2022 12:00AM,238482,10,Eywa Pharma Inc.,7.0,12.0,NaN
1,Jun 1 2022 12:00AM,238487,10,Eywa Pharma Inc.,3.0,1.0,NaN
2,Jun 1 2022 12:00AM,238489,10,Eywa Pharma Inc.,1.0,NaN,NaN
3,May 31 2022 5:03PM,238488,19,VITABIOTICS USA INC.,2.0,NaN,NaN
4,May 31 2022 4:36PM,238486,16,American International Chemical,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 1 2022 12:00AM,238482,10,Eywa Pharma Inc.,7.0,12.0,0.0
1,Jun 1 2022 12:00AM,238487,10,Eywa Pharma Inc.,3.0,1.0,0.0
2,Jun 1 2022 12:00AM,238489,10,Eywa Pharma Inc.,1.0,0.0,0.0
3,May 31 2022 5:03PM,238488,19,VITABIOTICS USA INC.,2.0,0.0,0.0
4,May 31 2022 4:36PM,238486,16,American International Chemical,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3099529,156.0,51.0,11.0
102,953807,3.0,1.0,0.0
12,715149,63.0,9.0,1.0
16,238486,1.0,0.0,0.0
18,715111,3.0,0.0,0.0
19,476972,3.0,0.0,0.0
20,953691,37.0,14.0,13.0
22,476961,2.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3099529,156.0,51.0,11.0
1,102,953807,3.0,1.0,0.0
2,12,715149,63.0,9.0,1.0
3,16,238486,1.0,0.0,0.0
4,18,715111,3.0,0.0,0.0
5,19,476972,3.0,0.0,0.0
6,20,953691,37.0,14.0,13.0
7,22,476961,2.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,156.0,51.0,11.0
1,102,3.0,1.0,0.0
2,12,63.0,9.0,1.0
3,16,1.0,0.0,0.0
4,18,3.0,0.0,0.0
5,19,3.0,0.0,0.0
6,20,37.0,14.0,13.0
7,22,2.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,156.0
1,102,Released,3.0
2,12,Released,63.0
3,16,Released,1.0
4,18,Released,3.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,102,12,16,18,19,20,22
Status,,,,,,,,
Completed,11.0,0.0,1.0,0.0,0.0,0.0,13.0,0.0
Executing,51.0,1.0,9.0,0.0,0.0,0.0,14.0,0.0
Released,156.0,3.0,63.0,1.0,3.0,3.0,37.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,102,12,16,18,19,20,22
0,Completed,11.0,0.0,1.0,0.0,0.0,0.0,13.0,0.0
1,Executing,51.0,1.0,9.0,0.0,0.0,0.0,14.0,0.0
2,Released,156.0,3.0,63.0,1.0,3.0,3.0,37.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,102,12,16,18,19,20,22
0,Completed,11.0,0.0,1.0,0.0,0.0,0.0,13.0,0.0
1,Executing,51.0,1.0,9.0,0.0,0.0,0.0,14.0,0.0
2,Released,156.0,3.0,63.0,1.0,3.0,3.0,37.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()